In [11]:
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import bs4

In [12]:
router_docs_loader = RecursiveUrlLoader("https://pub.dev/packages/flutter_bloc")

In [13]:
router_docs = router_docs_loader.load()
router_docs

[Document(metadata={'source': 'https://pub.dev/packages/flutter_bloc', 'content_type': 'text/html; charset="utf-8"', 'title': 'flutter_bloc | Flutter package', 'description': 'Flutter widgets that make it easy to implement the BLoC (Business Logic Component) design pattern. Built to be used with the bloc state management package.', 'language': 'en-us'}, page_content='<!DOCTYPE html>\n<html lang="en-us"><head><script src="https://www.googletagmanager.com/gtm.js?id=GTM-MX6DBN9" async="async"></script><script src="/static/hash-jc0slbjj/js/gtm.js" async="async"></script><meta charset="utf-8"/><meta http-equiv="x-ua-compatible" content="ie=edge"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="twitter:card" content="summary"/><meta name="twitter:site" content="@dart_lang"/><meta name="twitter:description" content="Flutter widgets that make it easy to implement the BLoC (Business Logic Component) design pattern. Built to be used with the bloc state management

In [14]:
# Splitting doc into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
chunked_docs = splitter.split_documents(router_docs)
chunked_docs

[Document(metadata={'source': 'https://pub.dev/packages/flutter_bloc', 'content_type': 'text/html; charset="utf-8"', 'title': 'flutter_bloc | Flutter package', 'description': 'Flutter widgets that make it easy to implement the BLoC (Business Logic Component) design pattern. Built to be used with the bloc state management package.', 'language': 'en-us'}, page_content='<!DOCTYPE html>'),
 Document(metadata={'source': 'https://pub.dev/packages/flutter_bloc', 'content_type': 'text/html; charset="utf-8"', 'title': 'flutter_bloc | Flutter package', 'description': 'Flutter widgets that make it easy to implement the BLoC (Business Logic Component) design pattern. Built to be used with the bloc state management package.', 'language': 'en-us'}, page_content='<html lang="en-us"><head><script src="https://www.googletagmanager.com/gtm.js?id=GTM-MX6DBN9" async="async"></script><script src="/static/hash-jc0slbjj/js/gtm.js" async="async"></script><meta charset="utf-8"/><meta http-equiv="x-ua-compatibl

In [15]:
# Vectorizing the parts and saving it in vector store
embedding = OllamaEmbeddings(
    model="llama3.1:8b"
)

db = FAISS.from_documents(documents=chunked_docs, embedding=embedding)

In [16]:
# Saving the store locally just in case 🥸
db.save_local("router_docs_vector")

In [17]:
# Querying the data to get answers
llm = ChatOllama(
    model="llama3.1:8b"
)

prompt_template = PromptTemplate.from_template("You are a professional reader who can read any given document and answer questions based on that. Here is the document: {document}, and this is the question: {question}. Only answer from the passed document context, do not include any other sources you were trained on, if you dont have the answer simply say Sorry and move on.")


In [21]:
query = "How can I make a counter app in green color?"

# Fetch relevant context from vector store
context_results = db.similarity_search(query=query)
context_results

## Connect all the results into a single string
final_ctx = ""

for res in context_results:
    final_ctx += res.page_content

final_ctx

'<a href="https://flutter.dev/docs/development/packages-and-plugins/favorites"><img src="https://raw.githubusercontent.com/felangel/bloc/master/assets/pub/flutter_favorite.png" width="100" alt="build"></a>\n</p>\n<p align="center">\n<img src="https://raw.githubusercontent.com/felangel/bloc/master/assets/logos/flutter_bloc.png" height="100" alt="Flutter Bloc Package">\n</p>\n<p align="center">\n<a href="https://pub.dev/packages/flutter_bloc"><img src="https://img.shields.io/pub/v/flutter_bloc.svg" alt="Pub"></a>\n<a href="https://github.com/felangel/bloc/actions" rel="ugc"><img src="https://github.com/felangel/bloc/workflows/build/badge.svg" alt="build"></a>),\n  ],\n  child: ChildA(),\n)\n</code></pre>\n<h2 id="gallery" class="hash-header">Gallery <a href="#gallery" class="hash-link">#</a></h2>\n<div>\n    <table>\n        <tbody><tr>\n            <td>\n                <a href="https://bloclibrary.dev/tutorials/flutter-counter" rel="ugc">\n                    <img src="https://raw.gith

In [22]:
chain = prompt_template | llm | StrOutputParser()

In [23]:
res = chain.invoke({'document' : final_ctx, 'question': query})
print(res)

Sorry! There's no direct information about creating a counter app in green color in the provided document. However, I can point to a tutorial that might be helpful.

According to the document, there is a tutorial available at this link: https://bloclibrary.dev/tutorials/flutter-counter It seems like it could provide guidance on building a counter app, but I couldn't find any specific information about using green color in it.
